#### Import standard Python modules

In [2]:
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

#### Import Azure ML SDK modules

In [3]:
import azureml.core
from azureml.core import Workspace, Environment
from azureml.core.model import Model, InferenceConfig
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

#### Check Azure ML SDK version

In [4]:
print(azureml.core.VERSION)

1.39.0


#### Create Azure ML Workspace

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()

AZ_SUBSCRIPTION_ID= "f0da6105-47f8-4fe9-8ca6-dd54a37a1fdc"

print(AZ_SUBSCRIPTION_ID)

ws = Workspace.create(name='ml-data-gurus',
                      subscription_id=AZ_SUBSCRIPTION_ID,
                    #   subscription_name='SpringSemester Maksad Annamuradov',
                      resource_group='azure-ml-rg',
                      create_resource_group=True,
                      location='westus2'
                     )

f0da6105-47f8-4fe9-8ca6-dd54a37a1fdc


Deploying StorageAccount with name mldatagustorage1f1c0be1b.
Deploying KeyVault with name mldatagukeyvault70fdd727.
Deployed StorageAccount with name mldatagustorage1f1c0be1b. Took 20.12 seconds.
Deployed KeyVault with name mldatagukeyvault70fdd727. Took 16.83 seconds.
Deploying Workspace with name ml-data-gurus.
Deploying AppInsights with name mldataguinsightsb16e26cf.
Deployed AppInsights with name mldataguinsightsb16e26cf. Took 22.23 seconds.
Deployed Workspace with name ml-data-gurus. Took 19.56 seconds.


In [10]:
# I have already created a resource group and a workspace in the Azure portal

ws = Workspace.get(name="ml-data-gurus",
               subscription_id=AZ_SUBSCRIPTION_ID,
               resource_group='azure-ml-rg')

#Or get from local config file
# ws = Workspace.from_config()
# ws.get_details()

#### Write configuration to local file

In [11]:
ws.write_config()

#### Create Azure ML Experiment

In [12]:
exp = Experiment(workspace=ws, name='firstexp')

#### Start logging metrics

In [7]:
run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

#### Load salary dataset

In [11]:
sal = pd.read_csv('data/sal.csv',header=0, index_col=None)
X = sal[['x']]
y = sal['y']

#### Split the train and test data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

#### Train the model

In [13]:
lm = LinearRegression() 
lm.fit(X_train,y_train) 

LinearRegression()

#### Freeze the model

In [14]:
filename = 'outputs/sal_model.pkl'
joblib.dump(lm, filename)

['outputs/sal_model.pkl']

#### Test the model

In [15]:
filename = 'outputs/sal_model.pkl'
loaded_model=joblib.load(filename)
y=loaded_model.predict([[21]])[0]
print(y)

141760.56910569107


C:\Users\maksa\Envs\azureml\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


#### Log metrics to Azure ML Experiment

In [13]:
run.log('Intercept :', lm.intercept_)
run.log('Slope :', lm.coef_[0])

#### End Azure ML Experiment

In [14]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

#### Get Portal URL

In [15]:
print(run.get_portal_url())

https://mlworkspace.azure.ai/portal/subscriptions/9be00a6f-5335-4d37-9847-2f7013522146/resourceGroups/mi2-stage/providers/Microsoft.MachineLearningServices/workspaces/salary/experiments/salexp/runs/d9aec621-6498-44be-991b-f2504ad47ab1


#### Register the model

In [16]:
model = Model.register(model_path = "outputs/sal_model.pkl",
                       model_name = "sal_model",
                       tags = {"key": "1"},
                       description = "Salary Prediction",
                       workspace = ws)

Registering model sal_model


#### Define Azure ML Deploymemt configuration

In [17]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Salary",  "method" : "sklearn"}, 
                                               description='Predict Stackoverflow Salary')

#### Create enviroment configuration file

In [18]:
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")

with open("salenv.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("salenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



#### Create Azure ML Scoring file

In [19]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sal_model')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


In [24]:
#Here is the link to the article on how to use environments
#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments

envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        #print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-PyTorch-1.3-CPU
Name AzureML-Dask-CPU
Name AzureML-Dask-GPU
Name AzureML-VowpalWabbit-8.8.0
Name AzureML-Triton
Name AzureML-minimal-ubuntu18.04-py37-cpu-inference
Name AzureML-mlflow-ubuntu18.04-py37-cpu-inference
Name AzureML-tensorflow-1.15-ubuntu18.04-py37-cpu-inference
Name AzureML-tensorflow-2.4-ubuntu18.04-py37-cpu-inference
Name AzureML-xgboost-0.9-ubuntu18.04-py37-cpu-inference
Name AzureML-pytorch-1.6-ubuntu18.04-py37-cpu-inference
Name AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu
Name AzureML-sklearn-0.24-ubuntu18.04-py37-cpu
Name AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11.0.3-gpu-inference
Name AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference
Name AzureML-onnxruntime-1.6-ubuntu18.04-py37-cpu-inference
Name AzureML-pytorch-1.7-ubuntu18.04-py37-cpu-inference
Name AzureML-sklearn-1.0-ubuntu20.04-py38-cpu
Name AzureML-tensorflow-2.6-ubuntu20.04-py38-cuda11-gpu
Name AzureML-tensorflow-2.5-ubuntu20.04-py38-cuda11-gpu
Name AzureML-tensorflow-2.4-ubuntu18.04-py37

#### Deploy the model to Azure Container Instance


In [25]:
env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [32]:
# From a Conda specification file
myenv = Environment.from_conda_specification(name = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu",
                                             file_path = "salenv.yml")

In [33]:
# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)


#### Expose web service

In [21]:
# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(
    workspace = ws,
    name = "salary-prediction",
    models = [model],
    inference_config = inference_config,
    deployment_config = aciconfig)

service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image salary-svc:1, operation "Succeeded"
Creating service
Running......................
SucceededACI service creation operation finished, operation "Succeeded"


#### Get the Web Service URL

In [36]:
print(service.scoring_uri)

http://410e3cdd-f93b-495a-a3d0-c86559d0770a.westus2.azurecontainer.io/score


In [26]:
!curl -X POST \
	-H 'Content-Type':'application/json' \
	-d '{"data":[[45]]}' \
	http://410e3cdd-f93b-495a-a3d0-c86559d0770a.westus2.azurecontainer.io/score

"[185924.7967479675]"

#### Delete Workspace and clean up resources

In [31]:
ws.delete()